In [1]:
import os, shutil, yaml, itertools, math, wandb
import numpy as np
import importlib
import os, re, math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import torch
nn = torch.nn
F = torch.nn.functional
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

from sprelnet import train

from sprelnet import util, wab, visualize, losses
from sprelnet.data import mnist, pixels
from sprelnet.networks.relations import get_relnet
from sprelnet.networks.unet import get_unet
from sprelnet.networks.adversarial import get_adv_sprelnet
from sprelnet.networks.contrastive import get_contra_net
from sprelnet.networks.patch_net import get_patch_net

ds_folder = '/data/vision/polina/scratch/clintonw/datasets'
multi_mnist_ds_path = os.path.join(ds_folder, 'multi-MNIST/ds_instance.bin')
pixel_ds_path = os.path.join(ds_folder, 'pixel_ds.bin')

In [ ]:
pixels.create_pixel_dataset();

In [ ]:
ds = util.load_binary_file(multi_mnist_ds_path)
ds["legend"]

In [ ]:
ds = util.load_binary_file(pixel_ds_path)
ds["datapoint loader"]= lambda dp: (dp.sum(0, keepdim=True).cuda(), (dp != 0).float().cuda())
util.save_binary_file(data=ds, path=pixel_ds_path)

In [ ]:
import importlib
importlib.reload(visualize)

## Interactive

In [18]:
class cmd_args:
    job_id = "rel_dbug"
    config_path = f"/data/vision/polina/users/clintonw/code/sprelnet/configs/{job_id}.yaml"
    p = f"{util.base_job_dir}/{job_id}"
    if not os.path.exists(p):
        os.makedirs(p)
with open(cmd_args.config_path, 'r') as stream:
    args = yaml.safe_load(stream)

run, args = train.prep_run(cmd_args, args)

if args["data loading"]["dataset"] == "MNIST grid":
    dataset = mnist.get_multi_mnist(run=run)
else:
    raise NotImplementedError("bad dataset")

dataloaders = util.get_dataloaders(dataset, batch_size=args["data loading"]["batch size"])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: clintonjwang (use `wandb login --relogin` to force relogin)


In [37]:
dataset = mnist.get_multi_mnist(run=run)
get_net_fxn = get_relnet
train_fxn = train.train_relnet

In [38]:
network = get_net_fxn(net_HPs=args["network"], dataset=dataset)
paths=args["paths"]
loss_settings=args["loss"]
optimizer_settings=args["optimizer"]

In [44]:
wab.log_relnet(network, dataset)

In [ ]:
network = get_relnet(net_HPs=args["network"], dataset=dataset)

train.train_relnet(paths=args["paths"],
    loss_settings=args["loss"], optimizer_settings=args["optimizer"],
    dataloaders=dataloaders, network=network, dataset=dataset)

In [ ]:
legend = mnist.get_multi_mnist_legend(key="index")
for label_pair in util.label_pairs_of_interest:
    img = visualize.get_multiscale_likelihood_kernel_composite(relnet, *label_pair)
    wandb.log({f"kernel ({legend[label_pair[0]]}-{legend[label_pair[1]]})": img})

In [ ]:
paths=args["paths"]
loss_settings=args["loss"]
optimizer_settings=args["optimizer"]

## Slurm

In [ ]:
sh script_train.sh unet_mni
sh script_train.sh adv_mni

sh script_train.sh patch_mni
sh script_train.sh van_dbug
sh script_train.sh van_mni
sh script_train.sh unet_pix
sh script_train.sh van_pix
sh script_train.sh rel_mni

In [ ]:
importlib.reload(util);

In [ ]:
util.end_slurm_jobs(["van_mni"])

In [4]:
#util.rename_job("rel_mni", "rel_mni_01")
util.delete_job_outputs("patch_mni")

In [ ]:
util.print_job_log("unet_pix")

In [ ]:
util.print_job_log("van_dbug")

In [ ]:
util.print_job_log("patch_mni")

In [ ]:
util.print_job_log("van_mni")

In [ ]:
wab.clear_failed_runs()

## Figures